In [42]:
import pandas as pd
import numpy as np
passing_df_raw = pd.read_csv('game_passing_df.csv')
receiving_df_raw = pd.read_csv('game_receiving_df.csv')
rushing_df_raw = pd.read_csv('game_rushing_df.csv')
DST_df_raw = pd.read_csv('DST.csv')
kicking_df_raw = pd.read_csv('kicking.csv')

In [43]:
kicking_df_raw.head()

,overallRank,player,team,position,bye,points,lower,upper,tier,ptSpread,risk,sleeper,ptsGame,adp,exp
0,72,Stephen Gostkowski,NE,K,11,161.24,149.50,169.57,1,20.07,2.33,NaN,10.08,112.53,12.0
1,80,Justin Tucker,BAL,K,10,158.32,134.27,171.44,1,37.17,3.40,NaN,9.90,124.02,6.0
2,83,Greg Zuerlein,LAR,K,12,156.54,115.00,174.67,1,59.67,3.47,NaN,9.78,116.68,6.0
3,90,Matt Bryant,ATL,K,8,153.60,135.81,164.85,1,29.04,3.27,NaN,9.60,139.37,16.0
4,104,Harrison Butker,KC,K,12,149.37,128.58,162.17,2,33.59,4.33,NaN,9.34,141.73,1.0


In [44]:
#formatting passing data
passing_df = passing_df_raw[['GameID', 'Passer_ID', 'Team', 'Player_Name', 'Total_Yards', 'Interceptions', 'TDs']]
passing_df.columns = ['Year', 'PlayerID', 'Team', 'Player_Name', 'Passing_Yards', 'Interceptions', 'Passing_TDs'] 

#formatting receiving data
receiving_df = receiving_df_raw[['GameID', 'Receiver_ID', 'Team', 'Player_Name', 'Receptions', 'Total_Yards', 'Fumbles', 'TDs']]
receiving_df.columns = ['Year', 'PlayerID', 'Team', 'Player_Name', 'Receptions', 'Receiving_Yards', 'Receiving_Fumbles', 'Receiving_TDs'] 

#formatting rushing data
rushing_df = rushing_df_raw[['GameID', 'Rusher_ID','Team', 'Player_Name', 'Total_Yards', 'Fumbles', 'TDs']]
rushing_df.columns = ['Year', 'PlayerID', 'Team', 'Player_Name', 'Rushing_Yards', 'Rushing_Fumbles', 'Rushing_TDs'] 

#formatting defensive data
DST_df = pd.DataFrame(DST_df_raw[['ptsGame']].values, index = DST_df_raw[['player']].values.flatten())

#formatting kicking data
kicking_df = pd.DataFrame(kicking_df_raw[['ptsGame']].values, index = kicking_df_raw[['player']].values.flatten())

In [45]:
#merging to one data frame 

#Columns: GameID, Team, Player_Name, Passing_Yards, Interceptions, Passing_TD's, Receptions, Receiving Yards, Receiving_Fumbles, Receiving_TDs, Rushing_Yards, Rushing_Fumbles, Rushing_TDs, 

a = passing_df
b = a.merge(rushing_df, 'outer')
c = b.merge(receiving_df, 'outer')

total_df = c

total_df = total_df.fillna(0)

In [46]:
total_df.head()

,Year,PlayerID,Team,Player_Name,Passing_Yards,Interceptions,Passing_TDs,Rushing_Yards,Rushing_Fumbles,Rushing_TDs,Receptions,Receiving_Yards,Receiving_Fumbles,Receiving_TDs
0,2009091000,00-0022924,PIT,B.Roethlisberger,363.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2009091000,00-0003292,TEN,K.Collins,244.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2009091300,00-0026143,ATL,M.Ryan,229.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2009091300,00-0019559,MIA,C.Pennington,182.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2009091300,00-0027020,MIA,P.White,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
scoring = {
    'Passing_Yards' : lambda x : x*.04,
    'Interceptions' : lambda x : -2.*x,
    'Passing_TDs' : lambda x : x*4., 
    'Rushing_Yards' : lambda x : x*.1,
    'Rushing_Fumbles' : lambda x : -2.*x,
    'Rushing_TDs' : lambda x : x*6.,
    'Receptions' : lambda x : x,
    'Receiving_Yards' : lambda x : x*.1,
    'Receiving_Fumbles' : lambda x : -2.*x,
    'Receiving_TDs' : lambda x : x*6.    
}

In [48]:
scoring_multiplier = [0.04, -2., 4., .1, -2., 6., 1, .1, -2., 6.]

In [49]:
df1 = total_df.iloc[:,4:]

total_df['Fantasy_Points'] = df1.mul(scoring_multiplier).sum(1)

In [50]:
#format gameID to just display year

total_df['Year'] = total_df['Year'].apply(lambda x: str(x)[:4])

total_df['Year'] = total_df['Year'].astype(int)

In [51]:
class Player(object):
    name = ""
    team = ""
    player_ID = ""

    # The class "constructor" - It's actually an initializer 
    def __init__(self, name, team, player_ID):
        self.name = name
        self.team = team
        self.player_ID = player_ID

def make_player(name, team, player_ID):
    player = Player(name, team, player_ID)
    return player


def make_team(team):
    tm = []
    for p in team:
        #format name
        plr = p.split()
        name = plr[0][0] + '.' + plr[1]
        #find team name and player ID
        year = 2017
        p = total_df['Player_Name'] == name
    
        y = total_df['Year'] == year

        s = total_df[p & y]
        
        if s.empty:
            player_ID = ''
            team = ''
            
        else:
            player_ID = s.iloc[0,1]
        
            team = s.iloc[0,2]
        
        plr = make_player(name, team, player_ID)
        
        tm.append(plr)
    
    return tm

In [52]:
class Fantasy_Team(object):
    name = ""
    team = ""
    win = 0
    sim = pd.DataFrame()


    # The class "constructor" - It's actually an initializer 
    def __init__(self, name, team, win, sim):
        self.name = name
        self.team = team
        self.win = win
        self.win = sim
        

def make_fantasy_team(name, team, win, sim):
    fantasy_team = Fantasy_Team(name, team, win, sim)
    return fantasy_team

In [53]:
#rosters

Ben_Roster = ['Carson Wentz', 
        'David Johnson', 
        'Keyan Drake', 
        'Julio Jones',
        'Larry Fitzgerald',
         'Travis Kelce',
           'Rashaad Penny']

Wes_Roster = ['Jared Goff',
             'Melvin Gordon',
             'Christian McCaffrey',
             'Antonio Brown',
             'Golden Tate',
             'Austin Seferian-Jenkins',
             'Jordan Howard']

Sam_Roster = ['Ben Roethlisberger',
             'Alvin Kamara',
             'Leonard Fournette',
             'Tyreek Hill',
             'Randall Cobb',
             'Evan Engram',
             'Devonta Freeman']

Connor_Roster = ['Tom Brady',
                'LeSean McCoy',
                'Alex Collins',
                'Marvin Jones Jr',
                'Robert Woods',
                'Rob Gronkowski',
                'Joe Mixon']

Devin_Roster = ['Russell Wilson',
               'Jerick McKinnon',
               'Jay Ajayi',
               'DeAndre Hopkins',
               'Odell Beckham Jr',
               'Kyle Rudolph',
               'Mike Evans']

Nate_Roster = ['Deshaun Watson',
              'Saquon Barkley',
              'Dalvin Cook',
              'Adam Thielen',
              'Jarvis Landry',
              'Jimmy Graham',
              'Royce Freeman']

Rob_Roster = ['Cam Newton',
             'Leveon Bell',
             'Kareem Hunt',
             'Keenan Allen',
             'Stefon Diggs',
             'Greg Olsen',
             'Chris Hogan']

JP_Roster = ['Drew Brees',
            'Ezekiel Elliot',
            'Derrick Henry',
            'Michael Thomas',
            'Devante Adams',
            'Delanie Walker',
            'T.Y. Hilton']

Austin_Roster = ['Andy Dalton',
                'Spencer Ware',
                'Theo Riddick',
                'A.J. Green',
                'Doug Baldwin',
                'George Kittle',
                'Demaryius Thomas']

Kory_Roster = ['Aaron Rodgers',
              'Todd Gurley',
              'Kerryon Johnson',
              'Amari Cooper',
              'Allen Robinson',
              'Zach Ertz',
              'Sammy Watkins']

In [54]:
#make team objects
Ben_Team = make_fantasy_team('Ben', make_team(Ben_Roster), 0, pd.DataFrame())

Wes_Team = make_fantasy_team('Wes', make_team(Wes_Roster), 0, pd.DataFrame())

Sam_Team = make_fantasy_team('Sam', make_team(Sam_Roster), 0, pd.DataFrame())

Connor_Team = make_fantasy_team('Connor', make_team(Connor_Roster), 0, pd.DataFrame())

Devin_Team = make_fantasy_team('Devin', make_team(Devin_Roster), 0, pd.DataFrame())

Nate_Team = make_fantasy_team('Nate', make_team(Nate_Roster), 0, pd.DataFrame())

Rob_Team = make_fantasy_team('Rob', make_team(Rob_Roster), 0, pd.DataFrame())

JP_Team = make_fantasy_team('JP', make_team(JP_Roster), 0, pd.DataFrame())

Austin_Team = make_fantasy_team('Austin', make_team(Austin_Roster), 0, pd.DataFrame())

Kory_Team = make_fantasy_team('Kory', make_team(Kory_Roster), 0, pd.DataFrame())

#Corrections

Connor_Team.team[4].player_ID = '00-0029293' #Correction due to many M.Jones in NFL in 2017
JP_Team.team[1].player_ID = '00-0033045' # No Ezekiel Elliot data in 2017
Austin_Team.team[1].player_ID = '00-0030414' #No Spencer Ware data in 2017

In [55]:
#adding defenses and kickers to rosters

class Defense_Kicker(object):
    name = ""
    points = 0


    # The class "constructor" - It's actually an initializer 
    def __init__(self, name, points):
        self.name = name
        self.points = points

def make_defense_kicker(name, points):
    defense = Defense_Kicker(name, points)
    return defense

In [56]:
#Defenses

Wes_Defense_Kicker = ['Saints', 'Ryan Succop']
Sam_Defense_Kicker = ['Vikings', 'Josh Lambo']
Ben_Defense_Kicker = ['Broncos', 'Justin Tucker']
Connor_Defense_Kicker = ['Eagles', 'Greg Zuerlein']
Devin_Defense_Kicker = ['Titans', 'Robbie Gould']
Nate_Defense_Kicker = ['Jaguars', 'Wil Lutz']
Rob_Defense_Kicker = ['Ravens', 'Matt Bryant']
JP_Defense_Kicker = ['Texans', 'Adam Vinatieri']
Austin_Defense_Kicker = ['Patriots', 'Chris Boswell']
Kory_Defense_Kicker = ['Rams', 'Stephen Gostkowski']

In [57]:
teams = [Ben_Team, Wes_Team, Sam_Team, Connor_Team, Devin_Team, Nate_Team, Rob_Team, JP_Team, Austin_Team, Kory_Team]
DST_kickers = [Ben_Defense_Kicker, Wes_Defense_Kicker, Sam_Defense_Kicker, Connor_Defense_Kicker, Devin_Defense_Kicker,
              Nate_Defense_Kicker, Rob_Defense_Kicker, JP_Defense_Kicker, Austin_Defense_Kicker, Kory_Defense_Kicker]

In [58]:
for team, DST_kicker in zip(teams, DST_kickers):
    #add defense
    defense_name = DST_kicker[0]
    defense_points = DST_df.loc[defense_name][0]
    team.team.append(make_defense_kicker(defense_name, defense_points))
    
    #add kicker
    kicker_name = DST_kicker[1]
    kicker_points = kicking_df.loc[kicker_name][0]
    team.team.append(make_defense_kicker(kicker_name, kicker_points))

In [59]:
def score_QB_RB_WR_TE(player, year):
    # Find the player and score them for the given week/year   
    p = total_df['PlayerID'] == player.player_ID
    
    y = total_df['Year'] == year

    s = total_df[p & y]
    
    if s.empty:
        mean = 10
        sd = 5
        return np.round(np.random.normal(mean, sd), 1) # scoring for rookies
    else:
        points = np.array(s['Fantasy_Points'])
        return  np.random.choice(points,1)

def score_DST_K(player):
    sd_kicking = 3
    sd_defense = 5
    if ' ' in player.name:
        return np.round(np.random.normal(player.points, sd_kicking), 1)
    else:
        return np.round(np.random.normal(player.points, sd_defense), 1)

In [60]:
import numpy as np
def get_score_for_player(player):
    
    # Sample the year and week
    year = np.random.choice([2016, 2017],
                            p=[.3,.7])
    
    try:
        return score_QB_RB_WR_TE(player, year)
    except AttributeError:
        return score_DST_K(player)

In [61]:
def simulate_season_points(team, exps=1):
    names = [p.name for p in team]
    scores = pd.DataFrame(data=np.zeros((exps,len(team))),
                          columns = names)
    for n in range(exps):
        for player in team:
            scores.loc[n,player.name] += get_score_for_player(player)
    scores['Total'] = scores[scores.columns].sum(axis=1)
    return scores

In [62]:
def determine_winner(team1, team2):
    
    team1.sim = simulate_season_points(team1.team)
    team2.sim = simulate_season_points(team2.team)
    
    #simulate defense and kickers points
    
    team1_total = team1.sim['Total'].values[0] 
    team2_total = team2.sim['Total'].values[0] 
    
    if team1_total  >= team2_total:
        team1.win +=1
    else:
        team2.win +=1

In [63]:
#Schedule
def run_season():
    
    #Initalize records to be zero
    Ben_Team.win = 0
    Wes_Team.win = 0
    Nate_Team.win = 0
    Sam_Team.win = 0
    Devin_Team.win = 0
    JP_Team.win = 0
    Kory_Team.win = 0
    Rob_Team.win = 0
    Connor_Team.win = 0
    Austin_Team.win = 0
    #Week 1
    determine_winner(Nate_Team, Wes_Team)
    determine_winner(Ben_Team, Devin_Team)
    determine_winner(Kory_Team, Rob_Team)
    determine_winner(Connor_Team, Sam_Team)
    determine_winner(JP_Team, Austin_Team)

    #Week 2

    determine_winner(Kory_Team, Ben_Team)
    determine_winner(Wes_Team, Connor_Team)
    determine_winner(Nate_Team, JP_Team)
    determine_winner(Devin_Team, Sam_Team)
    determine_winner(Rob_Team, Austin_Team)

    #Week 3

    determine_winner(JP_Team, Connor_Team)
    determine_winner(Sam_Team, Ben_Team)
    determine_winner(Austin_Team, Kory_Team)
    determine_winner(Devin_Team, Wes_Team)
    determine_winner(Rob_Team, Nate_Team)

    #Week 4

    determine_winner(Austin_Team, Sam_Team)
    determine_winner(Connor_Team, Devin_Team)
    determine_winner(JP_Team, Rob_Team)
    determine_winner(Ben_Team, Wes_Team)
    determine_winner(Kory_Team, Nate_Team)

    #Week 5

    determine_winner(Rob_Team, Devin_Team)
    determine_winner(Wes_Team, Sam_Team)
    determine_winner(Nate_Team, Austin_Team)
    determine_winner(Ben_Team, Connor_Team)
    determine_winner(Kory_Team, JP_Team)

    #Week 6

    determine_winner(Ben_Team, Nate_Team)
    determine_winner(Connor_Team, Kory_Team)
    determine_winner(Sam_Team, JP_Team)
    determine_winner(Devin_Team, Austin_Team)
    determine_winner(Wes_Team, Rob_Team)

    #Week 7

    determine_winner(Nate_Team, Connor_Team)
    determine_winner(Kory_Team, Sam_Team)
    determine_winner(JP_Team, Devin_Team)
    determine_winner(Austin_Team, Wes_Team)
    determine_winner(Rob_Team, Ben_Team)

    #Week 8

    determine_winner(Sam_Team, Nate_Team)
    determine_winner(Devin_Team, Kory_Team)
    determine_winner(Wes_Team, JP_Team)
    determine_winner(Ben_Team, Austin_Team)
    determine_winner(Connor_Team, Rob_Team)

    #Week 9

    determine_winner(Nate_Team, Devin_Team)
    determine_winner(Sam_Team, Wes_Team)
    determine_winner(Rob_Team, Kory_Team)
    determine_winner(Connor_Team, Ben_Team)
    determine_winner(Austin_Team, JP_Team)

    #Week 10

    determine_winner(Kory_Team, Wes_Team)
    determine_winner(Devin_Team, Ben_Team)
    determine_winner(JP_Team, Nate_Team)
    determine_winner(Sam_Team, Connor_Team)
    determine_winner(Austin_Team, Rob_Team)

    #Week 11

    determine_winner(JP_Team, Ben_Team)
    determine_winner(Connor_Team, Wes_Team)
    determine_winner(Kory_Team, Austin_Team)
    determine_winner(Sam_Team, Devin_Team)
    determine_winner(Nate_Team, Rob_Team)

    #Week 12

    determine_winner(Austin_Team, Connor_Team)
    determine_winner(Ben_Team, Sam_Team)
    determine_winner(Rob_Team, JP_Team)
    determine_winner(Wes_Team, Devin_Team)
    determine_winner(Nate_Team, Kory_Team)

    #Week 13

    determine_winner(Rob_Team, Sam_Team)
    determine_winner(Devin_Team, Connor_Team)
    determine_winner(Austin_Team, Nate_Team)
    determine_winner(Wes_Team, Ben_Team)
    determine_winner(JP_Team, Kory_Team) 

In [64]:
names = [Wes_Team.name, Sam_Team.name, Ben_Team.name, Connor_Team.name, Devin_Team.name, Nate_Team.name, 
         Rob_Team.name,JP_Team.name, Austin_Team.name, Kory_Team.name]


In [65]:
wins = [Wes_Team.win, Sam_Team.win, Ben_Team.win, Connor_Team.win, Devin_Team.win, Nate_Team.win, 
         Rob_Team.win, JP_Team.win, Austin_Team.win, Kory_Team.win]

In [66]:
def add_win(names, wins):
    for i, (win, team) in enumerate(zip(wins, names)):
        record[i, win] +=1

In [67]:
def make_csv(names, record):
    
    team_sim = []
    for i, (name) in enumerate(names):
            
        percentage = [x / num for x in record[i]]

        data = np.zeros((14, 3))
        data[:,0] = [int(i) for i in range(13,-1, -1)]
        data[:,1] = [int(i) for i in range(0,14)]
        data[:,2] = percentage[::-1]

        team_sim.append(pd.DataFrame(data=data, columns= ['W', 'L', 'Probability']))
    return team_sim

In [69]:
record = np.zeros((10, 14))
num = 150
for i in range(num):
    if i %10 ==0:
        print i
    run_season()
    wins = [Wes_Team.win, Sam_Team.win, Ben_Team.win, Connor_Team.win, Devin_Team.win, Nate_Team.win, 
         Rob_Team.win, JP_Team.win, Austin_Team.win, Kory_Team.win]
    
    add_win(names,wins)

    
team_sim = make_csv(names, record)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140


# Wesley

In [70]:
team_sim[0]

,W,L,Probability
0,13.0,0.0,0.000000
1,12.0,1.0,0.000000
2,11.0,2.0,0.026667
3,10.0,3.0,0.073333
4,9.0,4.0,0.113333
5,8.0,5.0,0.206667
6,7.0,6.0,0.220000
7,6.0,7.0,0.193333
8,5.0,8.0,0.093333
9,4.0,9.0,0.053333


# Sam

In [71]:
team_sim[1]

,W,L,Probability
0,13.0,0.0,0.000000
1,12.0,1.0,0.006667
2,11.0,2.0,0.026667
3,10.0,3.0,0.066667
4,9.0,4.0,0.160000
5,8.0,5.0,0.173333
6,7.0,6.0,0.233333
7,6.0,7.0,0.220000
8,5.0,8.0,0.086667
9,4.0,9.0,0.026667


# Ben

In [72]:
team_sim[2]

,W,L,Probability
0,13.0,0.0,0.000000
1,12.0,1.0,0.000000
2,11.0,2.0,0.000000
3,10.0,3.0,0.020000
4,9.0,4.0,0.126667
5,8.0,5.0,0.186667
6,7.0,6.0,0.200000
7,6.0,7.0,0.193333
8,5.0,8.0,0.166667
9,4.0,9.0,0.066667


# Connor

In [73]:
team_sim[3]

,W,L,Probability
0,13.0,0.0,0.000000
1,12.0,1.0,0.000000
2,11.0,2.0,0.000000
3,10.0,3.0,0.026667
4,9.0,4.0,0.013333
5,8.0,5.0,0.066667
6,7.0,6.0,0.140000
7,6.0,7.0,0.220000
8,5.0,8.0,0.180000
9,4.0,9.0,0.186667


# Devin

In [74]:
team_sim[4]

,W,L,Probability
0,13.0,0.0,0.000000
1,12.0,1.0,0.000000
2,11.0,2.0,0.033333
3,10.0,3.0,0.033333
4,9.0,4.0,0.133333
5,8.0,5.0,0.173333
6,7.0,6.0,0.246667
7,6.0,7.0,0.186667
8,5.0,8.0,0.113333
9,4.0,9.0,0.053333


# Nate

In [75]:
team_sim[5]

,W,L,Probability
0,13.0,0.0,0.000000
1,12.0,1.0,0.000000
2,11.0,2.0,0.013333
3,10.0,3.0,0.046667
4,9.0,4.0,0.126667
5,8.0,5.0,0.193333
6,7.0,6.0,0.233333
7,6.0,7.0,0.153333
8,5.0,8.0,0.133333
9,4.0,9.0,0.073333


# Rob

In [76]:
team_sim[6]

,W,L,Probability
0,13.0,0.0,0.006667
1,12.0,1.0,0.026667
2,11.0,2.0,0.066667
3,10.0,3.0,0.113333
4,9.0,4.0,0.260000
5,8.0,5.0,0.180000
6,7.0,6.0,0.180000
7,6.0,7.0,0.093333
8,5.0,8.0,0.046667
9,4.0,9.0,0.026667


# JP

In [77]:
team_sim[7]

,W,L,Probability
0,13.0,0.0,0.000000
1,12.0,1.0,0.006667
2,11.0,2.0,0.026667
3,10.0,3.0,0.093333
4,9.0,4.0,0.166667
5,8.0,5.0,0.173333
6,7.0,6.0,0.226667
7,6.0,7.0,0.160000
8,5.0,8.0,0.093333
9,4.0,9.0,0.040000


# Austin

In [78]:
team_sim[8]

,W,L,Probability
0,13.0,0.0,0.000000
1,12.0,1.0,0.000000
2,11.0,2.0,0.000000
3,10.0,3.0,0.000000
4,9.0,4.0,0.000000
5,8.0,5.0,0.006667
6,7.0,6.0,0.013333
7,6.0,7.0,0.033333
8,5.0,8.0,0.133333
9,4.0,9.0,0.166667


# Kory

In [79]:
team_sim[9]

,W,L,Probability
0,13.0,0.0,0.000000
1,12.0,1.0,0.000000
2,11.0,2.0,0.000000
3,10.0,3.0,0.020000
4,9.0,4.0,0.033333
5,8.0,5.0,0.106667
6,7.0,6.0,0.246667
7,6.0,7.0,0.220000
8,5.0,8.0,0.180000
9,4.0,9.0,0.120000


In [80]:
names = [Wes_Team.name, Sam_Team.name, Ben_Team.name, Connor_Team.name, Devin_Team.name, Nate_Team.name, 
         Rob_Team.name,JP_Team.name, Austin_Team.name, Kory_Team.name]

In [81]:
team_sim = make_csv(names, record)

In [82]:
overall = []
for team in team_sim:
    overall.append(sum(team['W'] * team['Probability']))

print np.round(overall, 1)
print names

[7.1 7.4 6.6 5.3 7.  6.9 8.2 7.4 3.1 6. ]
['Wes', 'Sam', 'Ben', 'Connor', 'Devin', 'Nate', 'Rob', 'JP', 'Austin', 'Kory']


In [83]:
a = pd.DataFrame(data=overall, columns= ['Expected Wins'], index = names)

a.sort_values(by=['Expected Wins'], ascending=False)

,Expected Wins
Rob,8.213333
JP,7.406667
Sam,7.360000
Wes,7.140000
Devin,6.986667
Nate,6.886667
Ben,6.600000
Kory,5.993333
Connor,5.320000
Austin,3.093333
